In [1]:
import numpy as np
import os
from sklearn.metrics import f1_score
import glob
import cv2
import scipy.ndimage
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from google.colab import drive
drive.mount('/mydrive', True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mydrive


In [0]:
def create_data(training_set):
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/Negative/
  !pwd
  path_read = '*.png'
  filename = glob.glob(path_read)
  count = 0
  for ii in filename:
    if count < 1000:
      img = cv2.imread(ii)
      training_set.append(img)
      count = count + 1
  
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/Positive/
  !pwd
  filename = glob.glob(path_read)
  count = 0
  for ii in filename:
    if count < 500:
      img = cv2.imread(ii)
      training_set.append(img)
      count = count + 1
  return training_set


In [0]:
def converted(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

In [0]:
def imfilter(img_, filter):
  result = np.zeros((img_.shape[0], img_.shape[1]))
  row = img_.shape[0]
  col = img_.shape[1]
  if (filter.shape[0] == np.transpose(filter).shape[0]): #1d array
    for i in range (row):
      for ii in range (col):
        if (ii == 0):
          temp = img_[i][ii] * filter[1] + img_[i][ii+1] * filter[2]
          result[i][ii] = temp
        elif (ii == col - 1):
          temp = img_[i][ii-1] * filter[0] + img_[i][ii] * filter[1]
          result[i][ii] =  temp
        else:
          temp = img_[i][ii-1] * filter[0] + img_[i][ii] * filter[1] + img_[i][ii+1] * filter[2]
          result[i][ii] =  temp
  else:
    for i in range (col):
      for ii in range (row):
        if (ii == 0):
          temp = img_[ii][i] * filter[1] + img_[ii+1][i] * filter[2]
          result[ii][i] = temp
        elif (ii == row - 1):
          temp = img_[ii-1][i] * filter[0] + img_[ii][i] * filter[1]
          result[ii][i] =  temp
        else:
          temp = img_[ii-1][i] * filter[0] + img_[ii][i] * filter[1] + img_[ii+1][i] * filter[2]
          result[ii][i] = temp
  return result
  



In [0]:
def getHistogram(magnitude, angles, numBins):
  magnitude = np.reshape(magnitude,(64,))
  angles = np.reshape(angles,(64,))
  H = np.zeros((numBins, ))#create 9 bins
  binSize = np.pi/numBins #each bin contains 20 degree, converted into rad
  #compute the histogram below
  cellSize_ = 64
  #dx = bin contains number in range of that degree
  d10 = (np.pi/180)*10 #10 degree in rad
  d30 = d10 + binSize
  d50 = d30 + binSize
  d70 = d50 + binSize
  d90 = d70 + binSize
  d110 = d90 + binSize
  d130 = d110 + binSize 
  d150 = d130 + binSize
  d170 = d150 + binSize

  for i in range (cellSize_ - 1):
    #convert angles into unsigned
    if (angles[i] < 0):
      angles[i] = angles[i] + np.pi
    #contribute elements into each bin  
    if (angles[i] >= 0  and angles[i] < d10):
                rightPortion = (angles[i] + d10 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[8] = H[8] + leftPortion 
                H[0] = H[0] + rightPortion
    elif  (angles[i] >= d10  and angles[i] < d30):
                rightPortion = (angles[i] - d10 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[0] = H[0] + leftPortion 
                H[1] = H[1] + rightPortion
    elif  (angles[i] >= d30  and angles[i] < d50):
                rightPortion = (angles[i] - d30 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[1] = H[1] + leftPortion 
                H[2] = H[2] + rightPortion
    elif  (angles[i] >= d50  and angles[i] < d70):
                rightPortion = (angles[i] - d50 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[2] = H[2] + leftPortion
                H[3] = H[3] + rightPortion
    elif  (angles[i] >= d70  and angles[i] < d90):
                rightPortion = (angles[i] - d70 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[3] = H[3] + leftPortion
                H[4] = H[4] + rightPortion
    elif  (angles[i] >= d90  and angles[i] < d110):
                rightPortion = (angles[i] - d90 )/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[4] = H[4] + leftPortion 
                H[5] = H[5] + rightPortion
    elif  (angles[i] >= d110 and angles[i] < d130):
                rightPortion = (angles[i]- d110)/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[5] = H[5] + leftPortion 
                H[6] = H[6] + rightPortion
    elif  (angles[i] >= d130 and angles[i] < d150):
                rightPortion = (angles[i]- d130)/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[6] = H[6] + leftPortion 
                H[7] = H[7] + rightPortion
    elif  (angles[i] >= d150 and angles[i] < d170):
                rightPortion = (angles[i] - d150)/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[7] = H[7] + leftPortion 
                H[8] = H[8] + rightPortion
    else:
                rightPortion = (angles[i] - d170)/binSize * magnitude[i]
                leftPortion  = magnitude[i] - rightPortion
                H[8] = H[8] + leftPortion 
                H[1] = H[0] + rightPortion
  return H
  

In [0]:
def getDescriptor(imgTemp):
  numBins = 9
  cellSize = 8
  descriptor = []
  counter  = 0
  numHorizon = 8 
  numVertical = 16
  hx = np.array([-1, 0, 1])
  hy = np.array([[-1], [0], [1]])
  dx = imfilter(imgTemp, hx)
  dy = imfilter(imgTemp, hy)
  dx = dx[1:np.shape(dx)[0] - 1, 1:np.shape(dx)[1] - 1]
  dy = dy[1:np.shape(dy)[0] - 1, 1:np.shape(dy)[1] - 1]
  angles = np.arctan2(dx, dy)
  magnit = np.zeros((128, 64))
  for i in range (128):
    for ii in range (64):
      magnit[i][ii] = np.sqrt(np.power(dx[i][ii], 2) + np.power(dy[i][ii], 2))
  for rr in range (numVertical - 1):
    row = (rr) * cellSize
    for cc in range (numHorizon - 1):
      col = (cc) * cellSize
      angBlock = angles[row:row+15, col:col+15]
      mgBlock = magnit[row:row+15, col:col+15]
      #mgBlock = np.multiply(mgBlock, mask)
      mgBlock1 = mgBlock[0:8,  0:8]
      mgBlock2 = mgBlock[0:8,  9:16] 
      mgBlock3 = mgBlock[9:16, 0:8] 
      mgBlock4 = mgBlock[9:16, 9:16]
      angBlock1 = angBlock[0:8 , 0:8]
      angBlock2 = angBlock[0:8 , 9:16]
      angBlock3 = angBlock[9:16, 0:8]
      angBlock4 = angBlock[9:16, 9:16]  
      hist1 = getHistogram(mgBlock1, angBlock1, numBins)
      hist2 = getHistogram(mgBlock1, angBlock1, numBins)
      hist3 = getHistogram(mgBlock1, angBlock1, numBins)
      hist4 = getHistogram(mgBlock1, angBlock1, numBins)
      blockHists = []
      blockHists.append(hist1)
      blockHists.append(hist2)
      blockHists.append(hist3)
      blockHists.append(hist4)
      blockHists = np.reshape(np.array(blockHists), (36,))
      descriptor.append(blockHists)
  return np.array(descriptor).ravel()
  
  

In [0]:
def preprocess(data_set):
  total_descriptor = []
  for ii in range (len(data_set)):
    data_set[ii] = converted(data_set[ii])
    buffer = getDescriptor(data_set[ii])
    total_descriptor.append(buffer)
  return np.array(total_descriptor)

In [0]:
def train_svm(descriptor, label):
  clf = svm.LinearSVC()
  clf.fit(descriptor, label)
  w = clf.coef_
  b = clf.intercept_
  w = w.ravel()
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/
  !pwd
  with open("w.txt",'w') as f:
    for a in w:
        print("%s" % a, file=f)
  print(num_lines)
  #print('w = ', w)
  print('b = ', b)
  return w, b

In [0]:
def get_w():
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/
  !pwd
  w = []
  f = open("w.txt", "r")
  for x in f:
    w.append(float(x))
  w = np.array(w).ravel()
  return w

In [0]:
def classify(img, b):
  w = get_w()
  score = 0
  for ii in range (3780):
    score = score + img[ii]*w[ii]
  score = score + b
  if (score > 0):
    print('YES')
    return True
  else:
    print('NO')
    return False


In [0]:
def execute(img, b):
	img = getDescriptor(img)
	classify(img, b)

In [0]:
def predict(b):
  total_descriptor = []
  label = []
  for i in range (12):
    label.append(0)
  for i in range (12):
    label.append(1)
  label = np.array(label).ravel()
  pred = []
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/Predict/
  !pwd
  path_read = '*.*'
  filename = glob.glob(path_read)
  pred_set = []
  for ii in filename:
    img = cv2.imread(ii)
    pred_set.append(img)
  total_descriptor = preprocess(pred_set)
  for ii in total_descriptor:
    classify(ii, b)
    if (classify(ii,b) == True):
      pred.append(1)
    else:
      pred.append(0)
  pred = np.array(pred).ravel()
  print ("f1_score:", f1_score(label, pred))
  


In [23]:
def main():
  b = -0.99264632
  predict(b)
  #train() neu can train thi goi ham nay
  #train = '/mydrive/My Drive/Images/training.png'  
 # train = '/mydrive/My Drive/Images/DSC_0373.JPG'  
  #train = cv2.imread(train) 
  #train = converted(train)
  #train_2 = cv2.imread(train_2)
  #train = getDescriptor(train)
  #classify(train,  b)
  #classify(train_2,  b)
if __name__ == "__main__":
  main()

/mydrive/My Drive/Machine Learning/Data_CK/Predict
/mydrive/My Drive/Machine Learning/Data_CK/Predict
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
NO
/mydrive

In [0]:
  %cd /mydrive/My\ Drive/Machine\ Learning/Data_CK/
  !pwd
  w = []
  f = open("w.txt", "r")
  for x in f:
    w.append(float(x))
  w = np.array(w).ravel()
  print(w)

/mydrive/My Drive/Machine Learning/Data_CK
/mydrive/My Drive/Machine Learning/Data_CK
[-3.78840334e-04 -8.49963008e-07  2.11580321e-05 ... -7.19811930e-05
 -6.23134985e-05  2.53732705e-07]
